<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/update_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")
install_package("tqdm")
install_package("xlrd>=2.0.1")
install_package("openpyxl")

In [73]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas

# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

#Players Update (fixed table)

In [62]:
list_urls=get_file_content('data/list_all_urls.json')
list_player_names=get_file_content('data/list_all_players_names.json')
fixed_table=get_file_content('data/df_player_fixed.csv')

url_ranking='https://www.atptour.com/en/rankings/singles?rankRange=1-200'

list_player_names_new,list_urls_new=players_from_url(url_ranking)
list_player_names_new=[k for k in list_player_names_new if k not in list_player_names]
list_urls_new=[k for k in list_urls_new if k not in list_urls]

df_new=create_new_fixed_table('', list_player_names_new, list_urls_new)



In [63]:
import pandas as pd

df_fixed_all=pd.concat([fixed_table,df_new])
list_player_names_all=list_player_names_new+list_player_names
list_urls_all=list_urls_new+list_urls

create_report_workflow(fixed_table,df_fixed_all,'reports/update_df_fixed.json',['Player Name'])

add_file_github('data/list_all_urls.json',str(list_urls_all))
add_file_github('data/list_all_players_names.json',str(list_player_names_all))
add_file_github('data/df_player_fixed.csv',df_fixed_all.to_csv(index=False))


# Ranking Update

In [77]:
# import datetime

previous_ranking=get_file_content("data/df_all_ranking.csv")

# Get the current date
current_date = datetime.today()
previous_week = current_date - timedelta(days=(7))

week_url=find_list_weekly_ranking_url(previous_week,current_date,nb_players_ranking=300)[0]
ranking_week=get_ranking(week_url)

df_ranking_all=pd.concat([previous_ranking,ranking_week]).reset_index(drop=True).drop_duplicates()
add_file_github("data/df_all_ranking.csv",df_ranking_all.to_csv(index=False))

create_report_workflow(previous_ranking,df_ranking_all,'reports/update_df_ranking.json',['player_name'])



<string>:49: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


# Odds Update

We use a brut force methodolodgy here, recreating the table from scratch every week.

In [84]:
# URL of the online-hosted Excel dataset
columns_selected=['Location','Tournament','Date','Winner','Loser']
df_all_betting=pd.DataFrame(columns=columns_selected)
for year in [2001+k for k in range(23)]:
  excel_url = "http://www.tennis-data.co.uk/"+str(year)+"/"+str(year)+".xlsx"

  # Read the Excel dataset into a Pandas DataFrame
  df = pd.read_excel(excel_url)

  odds_columns=[k for k in df.columns if k[-1]=='L' or k[-1]=='W']
  columns_selected_year=columns_selected+odds_columns
  # print(df.columns)
  df=df[columns_selected_year]

  # Display the first few rows of the DataFrame to verify the data
  df_all_betting=pd.concat([df_all_betting,df]).reset_index(drop=True)

create_report_workflow(get_file_content("data/df_all_betting.csv"),df_all_betting,'reports/update_df_betting.json',['Winner'])

add_file_github("data/df_all_betting.csv",df_all_betting.to_csv(index=False))


<string>:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.


SyntaxError: ignored

In [80]:
df_all_betting

,Location,Tournament,Date,Winner,Loser,CBW,CBL,GBW,GBL,IWW,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,Adelaide,AAPT Championships,2001-01-01,Clement A.,Gaudenzi A.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adelaide,AAPT Championships,2001-01-01,Goldstein P.,Jones A.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adelaide,AAPT Championships,2001-01-01,Haas T.,Smith L.,NaN,NaN,1.12,4.0,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adelaide,AAPT Championships,2001-01-01,Henman T.,Rusedski G.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelaide,AAPT Championships,2001-01-01,Hewitt L.,Arthurs W.,1.28,3.3,1.12,3.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60845,Tokyo,Japan Open Tennis Championships,2023-10-20,Karatsev A.,De Minaur A.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.91,1.51,2.67,1.47
60846,Tokyo,Japan Open Tennis Championships,2023-10-20,Mochizuki S.,Popyrin A.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.75,1.40,3.36,1.32
60847,Tokyo,Japan Open Tennis Championships,2023-10-21,Shelton B.,Giron M.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.58,2.72,1.53,2.52
60848,Tokyo,Japan Open Tennis Championships,2023-10-21,Karatsev A.,Mochizuki S.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.32,4.26,1.25,3.93
